<a href="https://colab.research.google.com/github/labra/RailDataForum_ValidationWorkshop/blob/main/RailDataForumExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rail data forum - Validation workshop examples

We will use [rdflib](https://rdflib.readthedocs.io/) to run the examples in Python

In [3]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 27.1 MB/s eta 0:00:00


In [25]:
from rdflib import Graph, Namespace

Create an example RDF graph

In [5]:
g = Graph()

In [6]:
person_ttl_str = """
prefix : <http://example.org/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:timbl a :Human ;
       :name       "Tim Berners-Lee" ;
       :birthPlace :london ;
       :birthDate  "1955-06-08"^^xsd:date ;
       :employer   :CERN ;
       :knows     :jose .

:london a :City, :Metropolis ;
        :country :UK .

:CERN a :Organization ;
      :name "CERN"    .

:jose :name       "Jose" ;
      :birthPlace :Oviedo .

:Oviedo a :City ;
        :country :Spain .
""";

Parse and load the example into the RDF graph

In [7]:
g.parse(data=person_ttl_str, format="ttl")

<Graph identifier=N6caa421ae062412fb3e9826f50b30577 (<class 'rdflib.graph.Graph'>)>

Now, we can do some SPARQL queries like finding the birth place and birth date of Tim Berners Lee.

In [8]:
query_str = """
PREFIX : <http://example.org/>

SELECT ?person ?name ?birthDate ?birthPlace ?country WHERE {
  ?person a :Human ;
    :name ?name ;
    :birthDate ?birthDate ;
    :birthPlace ?birthPlace .
  ?birthPlace :country ?country
}
""";

Run a SPARQL query

In [9]:
result = g.query(query_str);

In [11]:
for row in result:
    print(f"Name: {row.name}, birth date: {row.birthDate}, birthPlace: {row.birthPlace}")

Name: Tim Berners-Lee, birth date: 1955-06-08, birthPlace: http://example.org/london


RDF is very flexible and so, it can contain errors. For example, the folling RDF data contains some errors, like humans without a name, with more than 2 names, with a name that is a number, or that know something that is not a human.

In [16]:
rdf_with_errors_str ="""
prefix : <http://example.org/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:timbl a :Human ;
       :name       "Tim Berners-Lee" ;
       :birthPlace :london ;
       :birthDate  "1955-06-08"^^xsd:date ;
       :employer   :CERN ;
       :knows     :jose .

:london a :City, :Metropolis ;
        :country :UK .

:CERN a :Organization .

:jose :name       "Jose" ;
      :birthPlace :Oviedo .

:Oviedo a :City ;
        :country :Spain .

:alice a :Human ;
      :name 234 .

:bob a :Human ;
     :name "John" , "Jane" .

:carol a :Human ;
     :birthPlace :london .

:dave a :Human ;
     :name "John" ;
     :knows :CERN .
""";

In [17]:
g = Graph()

In [18]:
g.parse(data=rdf_with_errors_str, format="ttl")

<Graph identifier=Nf67b302ff38941c48524bd8fa14755f7 (<class 'rdflib.graph.Graph'>)>

As you can see, the RDF tools are quite permissive and don't complain about those errors following the principle that Any one can say Any thing about Any topic.

## Using SPARQL to validate

It is possible (but can be tedious) to validate the data using SPARQL. For example, the following query checks if there are nodes with a single value for name which isn't a string

In [19]:
sparql_check_str = """
PREFIX : <http://example.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?person WHERE {
  ?person a :Human ;
  {
    SELECT ?person ?name WHERE {
    ?person :name ?name .
    FILTER (!isLiteral(?name) || datatype(?name) != xsd:string)
    }
  } UNION {
    SELECT ?person (COUNT(?name) AS ?nameCount)
    WHERE {
        ?person a :Human .
        OPTIONAL { ?person :name ?name . }
    }
    GROUP BY ?person
    HAVING (COUNT(?name) != 1)
  }
}
""";


In [20]:
result = g.query(sparql_check_str);


In [21]:
for row in result:
    print(row)

(rdflib.term.URIRef('http://example.org/alice'),)
(rdflib.term.URIRef('http://example.org/bob'),)
(rdflib.term.URIRef('http://example.org/carol'),)


As an exercise, you can try to define another SPARQL query that checks that a human only knows other humans.

## Validating using ShEx

[ShEx](https://shex.io/) has been designed as a concise and human readable language to describe and validate RDF data. The following ShEx specifies a shape for humans:

In [22]:
person_schema = """
prefix : <http://example.org/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>

:Person {
 :name       xsd:string       ;
 :birthPlace @:Place        ? ;
 :birthDate  xsd:date       ? ;
 :employer   @:Organization * ;
 :knows      @:Person       *
}

:Place {
 :country . ?
}

:Organization {
    a [ :Organization ]
}
"""

In [23]:
!pip install pyshex

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 8.1 MB/s eta 0:00:00
  Created wheel for cfgraph: filename=CFGraph-0.2.1-py3-none-any.whl size=2214 sha256=08f746196eb3ac94640439e1c690d4df0614b8f8ff92bae1cf335cb59e69f0a4
  Stored in directory: /root/.cache/pip/wheels/8b/ac/8f/0f5b041561f10e6db39796e4ad11011b8f5a4b0a0ed74a5e61
Successfully built cfgraph


In [24]:
from pyshex import ShExEvaluator

In [26]:
ex = Namespace("http://example.org/")

In [28]:
nodes = [ ex.timbl, ex.alice, ex.bob, ex.carol, ex.dave ]

In [30]:
for node in nodes:
  rs = ShExEvaluator().evaluate(g, person_schema, focus=node, start=ex.Person)
  for r in rs:
    if r.result:
      print(f"Node {node} Passed as Person")
    else:
      print(f"Node {node} Failed as Person: {r.reason}")

Node http://example.org/timbl Passed as Person
Node http://example.org/alice Failed as Person:   Testing :alice against shape http://example.org/Person
    Datatype mismatch - expected: http://www.w3.org/2001/XMLSchema#string actual: http://www.w3.org/2001/XMLSchema#integer
Node http://example.org/bob Failed as Person:   Testing :bob against shape http://example.org/Person
    Triples:
      :bob :name "Jane" .
      :bob :name "John" .
   2 triples exceeds max {1,1}
Node http://example.org/carol Failed as Person:   Testing :carol against shape http://example.org/Person
       No matching triples found for predicate :name
Node http://example.org/dave Failed as Person:   Testing :dave against shape http://example.org/Person
    Testing :CERN against shape http://example.org/Person
         No matching triples found for predicate :name


## Validating with SHACL

In [31]:
shacl_shapes_str = """
prefix :       <http://example.org/>
prefix sh:     <http://www.w3.org/ns/shacl#>
prefix xsd:    <http://www.w3.org/2001/XMLSchema#>
prefix schema: <http://schema.org/>

:PersonShape a sh:NodeShape ;
   sh:targetClass :Human ;
   sh:property [
    sh:path      :name ;
    sh:minCount 1; sh:maxCount 1;
    sh:datatype xsd:string ;
  ] ;
  sh:property [
    sh:path :birthPlace ;
    sh:node :PlaceShape
  ] ;
  sh:property [
   sh:path     :birthDate ;
   sh:maxCount 1;
   sh:datatype xsd:date;
  ] ;
  sh:property [
    sh:path :employer ;
    sh:node :OrganizationShape
  ] ;
  sh:property [
    sh:path :knows ;
    sh:node :PersonShape
  ] .

:PlaceShape a sh:NodeShape ;
  sh:property [
    sh:path :country ;
    sh:nodeKind sh:IRI ;
    sh:minCount 1 ;
    sh:maxCount 1 ;
  ] .

:OrganizationShape a sh:NodeShape .
"""

In [32]:
!pip install pyshacl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 9.4 MB/s eta 0:00:00


In [33]:
from pyshacl import validate

In [34]:
shapes = Graph()

In [35]:
shapes.parse(data=shacl_shapes_str, format="ttl")

<Graph identifier=N43a1d79d203240bf925beb2a453bba3c (<class 'rdflib.graph.Graph'>)>

In [37]:
r = validate(g, shacl_graph = shapes)

In [38]:
conforms, results_graph, results_text = r
print(f"Conforms: {conforms}")
print(results_graph.serialize(format="turtle"))

Conforms: False
@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :bob ;
            sh:resultMessage "More than 1 values on :bob->:name" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxCountConstraintComponent ;
            sh:sourceShape _:n46a9a97b32fc413a92a6bb02cd0f0d03b1 ],
        [ a sh:ValidationResult ;
            sh:detail [ a sh:ValidationResult ;
                    sh:focusNode :CERN ;
                    sh:resultMessage "Less than 1 values on :CERN->:name" ;
                    sh:resultPath :name ;
                    sh:resultSeverity sh:Violation ;
                    sh:sourceConstraintComponent sh:MinCountConstraintComponent ;
                    sh:sourceShape _:n46a9a97b32fc413a92a6bb02cd0f0